# CP-SAT: Create booleans

In [1]:
from ortools.sat.python import cp_model

## Only enforce a constraint when a condition is verified

A great thing in CP SAT is being able to enable constraint only when some bool is true. 

However, this method doesn't work out of the box with linear expressions.

In [2]:
model = cp_model.CpModel()

x = model.NewIntVar(-100, 100, "x")
result = model.NewIntVar(0, 100, "result")

# We want to have result == x only if x >= 0
# Let's suppose x=10
model.Add(x == 10)
# The following will fail 
model.Add(result == x).OnlyEnforceIf(x >= 0)

TypeError: 'BoundedLinearExpression' object is not iterable

A trick is to use an intermediate boolean variable, with correct constraints.

The boolean variable will be true when the condition is enforced, and false otherwise. 

Here, we implement this for the condition "var is non zero", i.e. `var > 0`.

In [10]:
def create_boolean_is_positive(model:cp_model.CpModel, var: cp_model.IntVar):
    """Create a bool variable such that 
    If var >= 0 then bool = 1
    If var  < 0 then bool = 0 
    """
    boolean_var = model.NewBoolVar(name=var.Name() + "_is_positive")

    # Bool are casted to 0 if False and 1 if True, so you can do some operations with them
    model.Add(var >= 0).OnlyEnforceIf(boolean_var)
    model.Add(var < 0).OnlyEnforceIf(boolean_var.Not())

    return boolean_var

Let's try this function. If x is nonzero positive, we'd like to see the value of x as a result. 

In [11]:
model = cp_model.CpModel()

x = model.NewIntVar(-100, 100, "x")
result = model.NewIntVar(0, 100, "result")

# We want to have result == x only if x >= 0
# We create the intermediate variable 
x_is_positive = create_boolean_is_positive(model, x)

# Let's suppose x=10
model.Add(x == 10)
# The following will work
model.Add(result == x).OnlyEnforceIf(x_is_positive)

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Divide the result to get a rounded down solution
print(f"Solution is: {solver.Value(result)}")


Solution is: 10


Now if x is nonzero negative, we set the result to be 42. 

In [12]:
model = cp_model.CpModel()

x = model.NewIntVar(-100, 100, "x")
result = model.NewIntVar(0, 100, "result")
x_is_positive = create_boolean_is_positive(model, x)

# Let's suppose x=-10
model.Add(x == -10)
model.Add(result == x).OnlyEnforceIf(x_is_positive)
# And return 42 if x is not positive
model.Add(result == 42).OnlyEnforceIf(x_is_positive.Not())


# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Divide the result to get a rounded down solution
print(f"Solution is: {solver.Value(result)}")

Solution is: 42


Let's also inspect the value of the bool if x=0. In the function we decided for the bool to be true. This is indeed what we see. 

In [13]:
model = cp_model.CpModel()

x = model.NewIntVar(-100, 100, "x")
result = model.NewIntVar(0, 100, "result")
x_is_positive = create_boolean_is_positive(model, x)

# Let's test the case x==0 
model.Add(x == 0)

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Divide the result to get a rounded down solution
print(f"Solution is: {solver.Value(x_is_positive)}")

Solution is: 1


## Only enforce a constraint when var is equal to value

In [14]:
def create_boolean_is_equal_to(model:cp_model.CpModel, var: cp_model.IntVar, value: int):
    """Create a bool variable such that
    If var == value then bool = 1
    Else then bool = 0
    """
    boolean_var = model.NewBoolVar(name=f"{var.Name()}_is_equal_to_{value}")
    model.Add(value == var).OnlyEnforceIf(boolean_var)
    model.Add(value != var).OnlyEnforceIf(boolean_var.Not())

    return boolean_var

In [15]:
# TODO : test